#### Tasks under this project: 
- We're going to scrape Github topic url: 
- From list of topics. To Scrape Topic Title, Topic Description, Topic page URL
- For each topic we'll get the top 25 repos.
- For each topic, we'll get the repo name, username, stars & url.
- Stage1: For each topic we'll create a csv file in format
- Csv1: Topic title,Topic desc., Topic page url Top25
- Csv2: Topics Repo name, User name, Stars, repo URL
- Connecting script to MySQL database & load data on specific tables
#### Overview of tasks: we'll Extract(scrape) + Transform + Load Data(connect to a Database)

In [33]:
import requests
from bs4 import BeautifulSoup # importing beautifulsoup from bs4 module
import pandas as pd

In [34]:
topics_url= 'https://github.com/topics'

### Requests() library to crawl the webpage

In [81]:
response= requests.get(topics_url)

print(response.status_code)

200


In [37]:
# Content of page response.text
content= len(response.text)
content

185372

### Parsing the webpage

In [39]:
# creating a soup using bs4
soup= BeautifulSoup(response.content,'html.parser')
print(soup)


<!DOCTYPE html>

<html data-a11y-animated-images="system" data-a11y-link-underlines="true" data-color-mode="auto" data-dark-theme="dark" data-light-theme="light" lang="en">
<head>
<meta charset="utf-8"/>
<link href="https://github.githubassets.com" rel="dns-prefetch"/>
<link href="https://avatars.githubusercontent.com" rel="dns-prefetch"/>
<link href="https://github-cloud.s3.amazonaws.com" rel="dns-prefetch"/>
<link href="https://user-images.githubusercontent.com/" rel="dns-prefetch"/>
<link crossorigin="" href="https://github.githubassets.com" rel="preconnect"/>
<link href="https://avatars.githubusercontent.com" rel="preconnect"/>
<link crossorigin="anonymous" href="https://github.githubassets.com/assets/light-0eace2597ca3.css" media="all" rel="stylesheet"><link crossorigin="anonymous" href="https://github.githubassets.com/assets/dark-a167e256da9c.css" media="all" rel="stylesheet"><link crossorigin="anonymous" data-color-theme="dark_dimmed" data-href="https://github.githubassets.com/

### Now that we've all content Github topics page.. we make our path to target:
     

In [40]:
#Fetching Topics_title name under href topic links

topic_titles = soup.find_all('p', {'class': 'f3 lh-condensed mb-0 mt-1 Link--primary'})
#print(topic_titles)

Topic_title=[]
for i in topic_titles:
    t= i.text.strip()
    Topic_title.append(t)
print(Topic_title)

['3D', 'Ajax', 'Algorithm', 'Amp', 'Android', 'Angular', 'Ansible', 'API', 'Arduino', 'ASP.NET', 'Atom', 'Awesome Lists', 'Amazon Web Services', 'Azure', 'Babel', 'Bash', 'Bitcoin', 'Bootstrap', 'Bot', 'C', 'Chrome', 'Chrome extension', 'Command line interface', 'Clojure', 'Code quality', 'Code review', 'Compiler', 'Continuous integration', 'COVID-19', 'C++']


In [42]:
# Fetching topic_description
topic_des= soup.find_all('p', class_= 'f5 color-fg-muted mb-0 mt-1')
#print(topic_des)

Topic_desc=[]
for i in topic_des:
        x= i.text.strip()
        Topic_desc.append(x)
print(Topic_desc)

['3D refers to the use of three-dimensional graphics, modeling, and animation in various industries.', 'Ajax is a technique for creating interactive web applications.', 'Algorithms are self-contained sequences that carry out a variety of tasks.', 'Amp is a non-blocking concurrency library for PHP.', 'Android is an operating system built by Google designed for mobile devices.', 'Angular is an open source web application platform.', 'Ansible is a simple and powerful automation engine.', 'An API (Application Programming Interface) is a collection of protocols and subroutines for building software.', 'Arduino is an open source platform for building electronic devices.', 'ASP.NET is a web framework for building modern web apps and services.', 'Atom is a open source text editor built with web technologies.', 'An awesome list is a list of awesome things curated by the community.', 'Amazon Web Services provides on-demand cloud computing platforms on a subscription basis.', 'Azure is a cloud co

In [43]:
# Fetching topic Link
topic_links = soup.find_all('a', {'class': "no-underline flex-1 d-flex flex-column"})
#print(topic_titles)

base_url= 'https://github.com/topics'
Topic_link=[]

for i in topic_links:
    L= i.get('href')
    
    Topic_link.append(base_url + L[7:])
    

In [46]:
# Lets create a single function to scrape_topic: To extract topic_name, topic_url & topic_desc
Topic_title=[]
Topic_desc=[]
Topic_link=[]

def topic_name(soup):

# Fetching Topics_title name under href topic links
    topic_titles = soup.find_all('p', {'class': 'f3 lh-condensed mb-0 mt-1 Link--primary'})

    for i in topic_titles:
        t= i.text.strip()
        Topic_title.append(t)
    return Topic_title

def topic_desc(soup):    
    # Fetching topic_description
    topic_des= soup.find_all('p', class_= 'f5 color-fg-muted mb-0 mt-1')
    #print(topic_des)
    
    for i in topic_des:
        x= i.text.strip()
        Topic_desc.append(x)
    return Topic_desc

def topic_url(soup):
    # Fetching topic Link
    topic_links = soup.find_all('a', {'class': "no-underline flex-1 d-flex flex-column"})

    base_url= 'https://github.com/topics'

    for i in topic_links:
        L= i.get('href')
        Topic_link.append(base_url + L)
    return Topic_link
    
def scrape_topic():
    tp_url= 'https://github.com/topics'
    response= requests.get(tp_url)
    if response.status_code != 200:
        raise Exception('Failed loading the site : {tp_url}')
    else:
        topics_dict= {'Title': topic_name(soup),
                     'Description': topic_desc(soup),
                     'URL': topic_url(soup)}
        
        topic_df= pd.DataFrame(topics_dict)
        return topic_df.head() 
# Calling our function "scrape_topic()" & fetching topics info
scrape_topic()

,Title,Description,URL
0,3D,3D refers to the use of three-dimensional grap...,https://github.com/topics/topics/3d
1,Ajax,Ajax is a technique for creating interactive w...,https://github.com/topics/topics/ajax
2,Algorithm,Algorithms are self-contained sequences that c...,https://github.com/topics/topics/algorithm
3,Amp,Amp is a non-blocking concurrency library for ...,https://github.com/topics/topics/amphp
4,Android,Android is an operating system built by Google...,https://github.com/topics/topics/android


In [47]:
# Creating Dataframe using lists of inform. about Topics

topics_dict= {'Topic_title': Topic_title,
'Topic_desc': Topic_desc,
'Topic_link': Topic_link}
        
topic_df= pd.DataFrame(topics_dict)
topic_df 

,Topic_title,Topic_desc,Topic_link
0,3D,3D refers to the use of three-dimensional grap...,https://github.com/topics/topics/3d
1,Ajax,Ajax is a technique for creating interactive w...,https://github.com/topics/topics/ajax
2,Algorithm,Algorithms are self-contained sequences that c...,https://github.com/topics/topics/algorithm
3,Amp,Amp is a non-blocking concurrency library for ...,https://github.com/topics/topics/amphp
4,Android,Android is an operating system built by Google...,https://github.com/topics/topics/android
5,Angular,Angular is an open source web application plat...,https://github.com/topics/topics/angular
6,Ansible,Ansible is a simple and powerful automation en...,https://github.com/topics/topics/ansible
7,API,An API (Application Programming Interface) is ...,https://github.com/topics/topics/api
8,Arduino,Arduino is an open source platform for buildin...,https://github.com/topics/topics/arduino
9,ASP.NET,ASP.NET is a web framework for building modern...,https://github.com/topics/topics/aspnet


In [59]:
#topic_df['Topic_desc'][0:]
len(Topic_desc[0])

98

------

##### Now that we've data of Topics, we'll Write a function to fetch Repositories from individual Topic i.e a single topic has mutliple repos
##### we request each topic_url through looping and extract data into lists - finally convert to dataframe  

In [49]:

# creating Empty lists to store all the inform. about Repositories

repo_name = []
user_name = []
stars = []
Repo_link = []
Topic_name = []


def Repo_info(Topic_title): #argument "Topic_title" i.e list of topic names
 
        for topic in Topic_title: # we're trying to iterate through each Topic
                
                # for each cycle of requests our crawler fetches required info. from each Topic consecutively
                res = requests.get('https://github.com/topics/' + topic)
                
                # Converting the html content into a BeautifulSoup object 'sup'
                sup= BeautifulSoup(res.content, 'html.parser')


                # Picking out a bs4elemt from sup which contains info. about stars marked on Repos 
                star_tag= sup.find_all('span', {'class': 'Counter js-social-count'})

                # Fetching count stars(*) 
                for star in star_tag:
                        st= star.text
                        
                        if st == '' or st.isspace():
                                # Handles case where star count is 0
                                stars.append('0')
                        elif st[-1] == 'k':
                                st = int(float(st[:-1]) * 1000)
                                stars.append(str(st))
                        else:
                                stars.append(st)

                
                # Fetching The Repos_name & user_name from h3_tags which contains Repos_names, username. 
                h3_tags= sup.find_all('h3', {'class': 'f3 color-fg-muted text-normal lh-condensed'})
                for i in h3_tags:
                                
                        a= i.find_all('a')
                        ur_name= a[0].text.strip()
                        rep_name= a[1].text.strip()
                        user_name.append(ur_name)
                        repo_name.append(rep_name)

                        Topic_name.append(topic)  # creating a list/column Topic name for our final dataframe

                # Fetching The Repository links from repo_linktag which contains repo_urls
                repo_linktag = sup.find_all('a', {'class' : 'Link text-bold wb-break-word'})

                l= 'https://github.com/topics'
                for i in repo_linktag:
                        links= l + i.get('href')
                        Repo_link.append(links)

Repo_info(Topic_title)
                              
print(user_name)
print(repo_name)
print(stars)
print(Repo_link)
print(Topic_name)

['mrdoob', 'pmndrs', 'libgdx', 'BabylonJS', 'ssloy', 'FreeCAD', 'lettier', 'aframevr', 'CesiumGS', 'blender', 'MonoGame', 'isl-org', 'metafizzy', 'timzhang642', 'nerfstudio-project', 'a1studmuffin', 'domlysz', 'FyroxEngine', 'google', 'openscad', 'ljianshu', 'metafizzy', 'olifolkerd', 'developit', 'jquery-form', 'Studio-42', 'elbywan', 'dwyl', 'ded', 'wendux', 'LeaVerou', 'noelboss', 'craftpip', 'k8w', 'nette', 'taoensso', 'loadingio', 'joaomilho', 'codingforentrepreneurs', 'u014427391', 'jwasham', 'trekhleb', 'TheAlgorithms', 'CyC2018', 'yangshun', 'krahets', 'kdn251', 'TheAlgorithms', 'azl397985856', 'youngyangyang04', 'algorithm-visualizer', 'huihut', 'halfrost', 'TheAlgorithms', 'TheAlgorithms', 'donnemartin', 'crossoverJie', 'keon', 'mxgmn', 'trekhleb', 'milligram', 'ampproject', 'AlexHex7', 'wprig', 'ampproject', 'GoogleForCreators', 'EmbeddedRPC', 'DeepVAC', 'xtrime-ru', 'ampproject', 'Azure', 'iheartradio', 'juusaw', 'NVlabs', 'wildhaber', 'Codeinwp', 'nuxt-community', 'nxp-mcu

In [50]:
# Now that our lists contain data we create Dataframe to structure our data in tabular
repoinfo_dict = {'Repo_Name': repo_name, 'User_name': user_name, 'No_of_Stars': stars, 
                        'Repo_link': Repo_link, 'Topic_Name': Topic_name}

Repos_df= pd.DataFrame(repoinfo_dict, index= None)
Repos_df

,Repo_Name,User_name,No_of_Stars,Repo_link,Topic_Name
0,three.js,mrdoob,98200,https://github.com/topics/mrdoob/three.js,3D
1,react-three-fiber,pmndrs,25600,https://github.com/topics/pmndrs/react-three-f...,3D
2,libgdx,libgdx,22600,https://github.com/topics/libgdx/libgdx,3D
3,Babylon.js,BabylonJS,22200,https://github.com/topics/BabylonJS/Babylon.js,3D
4,tinyrenderer,ssloy,19100,https://github.com/topics/ssloy/tinyrenderer,3D
...,...,...,...,...,...
575,awesome-coronavirus,soroushchehresa,1600,https://github.com/topics/soroushchehresa/awes...,COVID-19
576,CoronaTracker,mhdhejazi,1500,https://github.com/topics/mhdhejazi/CoronaTracker,COVID-19
577,Face-Mask-Detection,chandrikadeb7,1500,https://github.com/topics/chandrikadeb7/Face-M...,COVID-19
578,tika-python,chrismattmann,1400,https://github.com/topics/chrismattmann/tika-p...,COVID-19


In [77]:

Repos_df.loc[Repos_df['No-of-Stars'] == '', :]

,Repo_Name,User_name,No-of-Stars,Repo_link,Topic_Name


In [ ]:
Repos_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 580 entries, 0 to 579
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Repo_Name    580 non-null    object
 1   User_name    580 non-null    object
 2   No-of-Stars  580 non-null    object
 3   Repo_link    580 non-null    object
 4   Topic_Name   580 non-null    object
dtypes: object(5)
memory usage: 22.8+ KB


### Connecting to MySQL database & loading scraped data to mysql db
We've two options preferable to connect to our Relational database:
1. mysql.connector:- can only connect MySQL dbms
2. sqlalchemy ORM:- we'll go with sqlalchemy ORM as it allows to connect with difft. RDBMS like (Postgres/Mysql/sqlserver) 

In [78]:
import pandas as pd
from sqlalchemy import create_engine, Column, Integer, String, PrimaryKeyConstraint, ForeignKey
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import relationship, sessionmaker
#  SQLAlchemy library sqlalchemy.ext.declarative Declarative base 

# In SQLALCHEMY we deine "Python Classe" as "Database TABLES" with Schema

base = declarative_base() # every table we define will inherit from this base class

#Defining our Topics_table 

class Topics_table(base):
     __tablename__ = 'Topics_table'
     
     Topic_title = Column(String(100), primary_key = True)  
     Topic_desc = Column(String(200))
     Topic_link = Column(String(150))
     
     # One-to-many relationship with Repos_table
     repos = relationship("Repos_table", back_populates="topic")

# Defining Repos_table

class Repos_table(base):
     __tablename__ = 'Repos_table'
     
     # Define an auto-increment integer primary key column
     id = Column(Integer, primary_key=True, autoincrement=True)       
     # for sqlalchemy orm its easier to manipulate objects & perform CRUD if each table has primary-key defined. if not defined it often pops error
     User_name = Column(String(100))
     No_of_Stars = Column(String(100))
     Repo_link = Column(String(150))
     Topic_Name = Column( String(100), ForeignKey('Topics_table.Topic_title')) 

     # Many-to-one relationship with Topics_table
     topic = relationship("Topics_table", back_populates="repos")

     # MySQL connection settings
db_user = 'root'
db_password = '12345'
db_host = 'localhost'
db_port = '3306'
db_name = 'Github_db'

# Create MySQL connection string
connection_string = f'mysql+pymysql://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}'



C:\Users\hp\AppData\Local\Temp\ipykernel_10264\1061425340.py:8: MovedIn20Warning: The ``declarative_base()`` function is now available as sqlalchemy.orm.declarative_base(). (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  base = declarative_base() # every table we define will inherit from this base class


In [79]:
# Creating SQLAlchemy engine & our sqlalchemy classes/database table will be created immediately
engine = create_engine(connection_string)

# Create tables in the database
base.metadata.create_all(engine)

#table_name = 'your_table_name'
#Repos_df.to_sql(table_name, con=engine, if_exists='append', index=False)


In [80]:
# Now mapping data from Datframe to our SQL objects/ Inserting data into our tables

# Create SQLAlchemy session
Session = sessionmaker(bind=engine)
session = Session()

# Map data from Topics_df to Topics_table
for index, row in topic_df.iterrows():
    topic = Topics_table(
        Topic_title=row['Topic_title'],
        Topic_desc=row['Topic_desc'],
        Topic_link=row['Topic_link']
    )
    session.add(topic)
    
# Map data from Repos_df to Repos_table
for index, row in Repos_df.iterrows():
    repo = Repos_table(
        User_name=row['User_name'],
        No_of_Stars=row['No_of_Stars'],
        Repo_link=row['Repo_link'],
        Topic_Name=row['Topic_Name']  # Assuming 'Topic_name' is present in Repos_df
    )
    session.add(repo)

# Commit changes
session.commit()
session.close()

##### Thankyou